In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
pd.set_option('display.width', 15000)
pd.set_option('display.max_columns', 100)
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context('poster')
%matplotlib inline

In [2]:
class id2num:
    def __init__(self):
        pass
    
    def _id2num_dict(self, df, col):
        _num2id = df[col].values
        _id2num = {v:n for n, v in enumerate(_num2id)}
        return _num2id, _id2num
    
    def _id2num(self, df, col, i2n):
        def transform(v):
            return i2n[v]
        df[col] = df[col].apply(transform)
        return df
    
    def fit_transform(self, dfr, dfb, dfu):
        bus_n2i, bus_i2n = self._id2num_dict(dfb, 'business_id') 
        user_n2i, user_i2n = self._id2num_dict(dfu, 'user_id')
        dfb = self._id2num(dfb, 'business_id', bus_i2n)
        dfu = self._id2num(dfu, 'user_id', user_i2n)
        dfr = self._id2num(dfr, 'business_id', bus_i2n)
        dfr = self._id2num(dfr, 'user_id', user_i2n)
        return dfr, dfb, dfu

In [3]:
# Load data and convert user_id and review_id to integers

raw_dir = 'raw/'
df_business = pd.read_csv(raw_dir + 'business.csv')
df_review = pd.read_csv(raw_dir + 'review.csv')
df_user = pd.read_csv(raw_dir + 'user.csv')

df_review, df_business, df_user = id2num().fit_transform(df_review, df_business, df_user)
data_review = df_review[['user_id', 'business_id', 'stars']]

In [4]:
print('The number of missing values in review.stars is {}.'.format(np.sum(df_review.stars.isnull())))

The number of missing values in review.stars is 0.


In [5]:
# check duplicates

df_review.groupby(['user_id', 'business_id'], as_index=False).count()\
[df_review.groupby(['user_id', 'business_id'], as_index=False).count() != 1].dropna()[['user_id', 'business_id', 'stars']]

,user_id,business_id,stars
1773114,98431.0,63954.0,2.0


In [6]:
df_review.query('user_id == 98431 & business_id == 63954')

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
4291340,0,98431,1l-mU-BwQsZRW7BKdHVwZw,"This place continues to be a shit hole, with P...",63954,1,2017-05-02,0,0
4291341,0,98431,IcpPtd37iJEthZybuasS4g,My mother recently spent a good deal of time i...,63954,1,2015-02-06,9,0


As we can see, there is only one case where a user rate the same restaurant more than once and the ratings are the same in this case. We can simply drop the duplicate.

In [7]:
df_review = df_review.drop(4291341)
df_review.query('user_id == 98431 & business_id == 63954')

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
4291340,0,98431,1l-mU-BwQsZRW7BKdHVwZw,"This place continues to be a shit hole, with P...",63954,1,2017-05-02,0,0


In this project, we are only interested in restaurants; so we remove barbershops.

In [8]:
def remove_barbershop(df_business=df_business, df_review=df_review, df_user=df_user):
    b_cols = [col for col in df_business.columns if 'HairSpecializesIn' in col]
    df_business = df_business[np.all(df_business[b_cols].isnull(), axis=1)].drop(156300)
    for col in b_cols:
        del df_business[col]
    df_review = pd.merge(df_business[['business_id']], df_review, how='left', on='business_id')
    data_review = df_review[['user_id', 'business_id', 'stars']]
    df_user = pd.merge(pd.DataFrame(data_review['user_id'].unique(), \
                                    columns=['user_id']), df_user, how='left', on='user_id')
    return df_business, df_review, df_user, data_review

In [9]:
df_business, df_review, df_user, data_review = remove_barbershop()

In [11]:
# Save data
data_dir = 'data/Full/'
df_review.to_csv(data_dir + 'review.csv', index=False)
df_business.to_csv(data_dir + 'business.csv', index=False)
df_user.to_csv(data_dir + 'user.csv', index=False)
data_review.to_csv(data_dir + 'data_review.csv', index=False)

In [12]:
df_business[['city', 'name']].groupby('city', as_index=False).count().sort_values(by='name', ascending=False)

,city,name
394,Las Vegas,24497
654,Phoenix,15527
907,Toronto,15338
134,Charlotte,7476
804,Scottsdale,7365
673,Pittsburgh,5648
524,Montréal,5160
489,Mesa,5107
315,Henderson,4072
898,Tempe,3914


In [13]:
def get_city(city, df_business=df_business, df_review=df_review, df_user=df_user):
    dfb = df_business[df_business['city'] == city]
    dfr = pd.merge(dfb[['business_id']], df_review, how='left', on='business_id')
    datar = dfr[['user_id', 'business_id', 'stars']]
    dfu = pd.merge(pd.DataFrame(datar['user_id'].unique(), columns=['user_id']), df_user, how='left', on='user_id')
    return dfb, dfr, dfu, datar

In [14]:
dfb, dfr, dfu, datar = get_city('Las Vegas')
print(dfr.shape)
data_dir = 'data/Las_Vegas/'
dfr.to_csv(data_dir + 'review.csv', index=False)
dfb.to_csv(data_dir + 'business.csv', index=False)
dfu.to_csv(data_dir + 'user.csv', index=False)
datar.to_csv(data_dir + 'data_review.csv', index=False)

(1447796, 9)


In [15]:
dfb, dfr, dfu, datar = get_city('Champaign')
print(dfu.shape)
data_dir = 'data/Champaign/'
dfr.to_csv(data_dir + 'review.csv', index=False)
dfb.to_csv(data_dir + 'business.csv', index=False)
dfu.to_csv(data_dir + 'user.csv', index=False)
datar.to_csv(data_dir + 'data_review.csv', index=False)

(9082, 22)
